# Differential Counts

I take code from 03.2017_SDSS_S82_Hess_diagrams,  to make all more manageable.

Anything here is meant to directly reproduce Fig.1  from Ivezic+2004 : differential counts of objects, selected by color cuts, or chi2 and color, cuts. 

## Read-in the data 

In [2]:
import pandas as pd
pd.options.display.max_columns = 999
from itertools import product
import numpy as np
import os 
import sys
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from matplotlib import rcParams

from scipy.stats import binned_statistic_2d
from astropy.table import Table
from astropy.table import vstack
from astropy.table import Column

import datetime

In [ ]:
%time 
# NCSA 
# using astropy Table : discussion about speed 
# https://github.com/astropy/astropy/issues/3334 

# In the last resort : line-by-line ascii approach
# http://hea-www.harvard.edu/~aldcroft/tmp/p4a/hamogu/html/files/asciifiles.html
    
# old processing ( using old error even for faint objects, instead of 
# using the new faintRMS ).  
# fname1 =  'Var_ugriz_11_patches_NCSA_narrow.csv' 
# fname2 =  'Var_ugriz_11_patches_IN2P3_narrow.csv.gz' 

fname1 = 'VarD_ugriz_11_patches_NCSA_narrow.csv.gz'
fname2 = 'VarD_ugriz_11_patches_IN2P3_narrow.csv.gz'

DirIn = '../data_products/varMetricsMerged/'


nrows = None
if nrows  : 
    # NCSA
    data1_df = pd.read_csv(DirIn+fname1, compression='gzip', 
                     nrows=5e6 )
    data1 = Table.from_pandas(data1_df)
    # IN2P3
    data1_df = pd.read_csv(DirIn+fname1, compression='gzip', 
                     nrows=5e6 )
    data1 = Table.from_pandas(data1_df)
    
else : 
    data1 = Table.read(DirIn+fname1, format='csv')
    data2 = Table.read(DirIn+fname2, format='csv')

    
    
#data1 = Table.read(DirIn+fname1, format='csv')

